# Analysis

In [1]:
%reset -f

# Genutzte Umgebung

In [2]:
import pandas as pd
from logger import Logger
from test_executer import TestExecutor
import extract
import sys
import numpy as np

logger = Logger()
testExecutor = TestExecutor(logger)

dependencies = [
    ('system', sys.version_info),
    ('pandas', pd.__version__),
    ('numpy', np.__version__),
]
for dependency in dependencies:
    logger.log(f"{dependency[0]} is installed with version {dependency[1]}")

{"type": "info", "time": 1657173615565, "message": "system is installed with version sys.version_info(major=3, minor=9, micro=12, releaselevel='final', serial=0)", "params": null}
{"type": "info", "time": 1657173615565, "message": "pandas is installed with version 1.4.1", "params": null}
{"type": "info", "time": 1657173615565, "message": "numpy is installed with version 1.20.3", "params": null}


## Setup der Daten

Zuerst laden wir die benötigten Daten herunter und initialisieren die genutzten Python Objekte.

In [3]:
tables = [
    "careplans",
    "conditions",
    "observations",
    "patients",
]

files = [
    "data/others/",
    "data/asthma/",
    "data/gallstones/",
    "data/hypertension/",
]

In [4]:
!mkdir -p data/allergy

Syntaxfehler.


In [5]:
from urllib.request import urlopen
import os

def ensure_file_has_been_downloaded(filename):
    full_filename = "../" + filename

    url = "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/" + filename
    if os.path.isfile(full_filename):
        logger.log("File {} already exists, skipping download".format(filename))
    else:
        logger.log("Downloading {}".format(filename))
        download_file(url, full_filename)

def download_file(url, filename):
    with open(filename, 'wb') as out_file:
        with urlopen(url) as file:
            out_file.write(file.read())

if not os.path.isfile("extract.py"):
    download_file(
        "https://raw.githubusercontent.com/Fuenfgeld/DMA2022DataProjectC/main/src/extract.py",
        "extract.py"
    )

for file in files:
    for table in tables:
        ensure_file_has_been_downloaded(file+table+".csv")

{"type": "info", "time": 1657173615988, "message": "File data/others/careplans.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615991, "message": "File data/others/conditions.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615991, "message": "File data/others/observations.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615992, "message": "File data/others/patients.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615992, "message": "File data/asthma/careplans.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615992, "message": "File data/asthma/conditions.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615993, "message": "File data/asthma/observations.csv already exists, skipping download", "params": null}
{"type": "info", "time": 1657173615993, "message

## Mit Datenbank verbinden

In [6]:
databaseFile = "data.sqlite"

logger.startTimeMeasurement('open-db', 'Connected to db and created tables')
connection = extract.connect_to_db(logger, databaseFile)  # create table patients, observations, conditions, careplans
logger.endTimeMeasurement('open-db')

In [7]:
def test_sqliteConnection(_logger):
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
    tablesInDb = list(map(lambda tableResult: tableResult[0], cursor.fetchall()))
    tablesInDb.sort()

    for table in tables:
        if not(table in tablesInDb):
            raise Exception('Table not found:', table)

testExecutor.execute('Test connection to database', test_sqliteConnection)

{"type": "info", "time": 1657173616264, "message": "✅ Test ran successfully: Test connection to database", "params": null}


## Daten in Datenbank laden

Lade der verwendete Daten in die Datenbank:

-   careplans
-   conditions
-   observations
-   patients

In [8]:
logger.startTimeMeasurement('load-data', 'Loading data into db')
for file in files:
    for table in tables:
        extract.insert_values_to_table(logger, connection.cursor(), table, "../"+ file + table + ".csv")  # TODO: insert ALL values in the right tables 
        connection.commit()

logger.endTimeMeasurement('load-data')

{"type": "info", "time": 1657173616383, "message": "🏗 Extracting data from ../data/others/careplans.csv", "params": null}
{"type": "info", "time": 1657173616398, "message": "🏗 Extracting data from ../data/others/conditions.csv", "params": null}
{"type": "info", "time": 1657173616459, "message": "🏗 Extracting data from ../data/others/observations.csv", "params": null}
{"type": "info", "time": 1657173617259, "message": "🏗 Extracting data from ../data/others/patients.csv", "params": null}
{"type": "info", "time": 1657173617266, "message": "🏗 Extracting data from ../data/asthma/careplans.csv", "params": null}
{"type": "info", "time": 1657173617289, "message": "🏗 Extracting data from ../data/asthma/conditions.csv", "params": null}
{"type": "info", "time": 1657173617384, "message": "🏗 Extracting data from ../data/asthma/observations.csv", "params": null}
{"type": "info", "time": 1657173620199, "message": "🏗 Extracting data from ../data/asthma/patients.csv", "params": null}
{"type": "info", "

## Messung der Datenfehler

Für unsere Forschungsfrage sind nur alle Daten mit gemessenen BMI relevant. Wurde dieser nicht vermessen oder eingetragen können die Daten für die Forschungsfrage nicht verwendet werden und sind somit unbrauchbar.

In [9]:
all_patients_query = """
SELECT COUNT(id) FROM patients;"""
count_bmi_query = """
SELECT COUNT(distinct id) FROM patients JOIN observations on patients.id == observations.patient WHERE observations.Code = '59576-9'"""

count_all_bmi_query = f"""
SELECT COUNT(patient) FROM observations WHERE observations.Code = '59576-9'"""

patient_all_count = connection.execute(all_patients_query).fetchall()[0][0]
patient_bmi_count = connection.execute(count_bmi_query).fetchall()[0][0]
bmi_count = connection.execute(count_all_bmi_query).fetchall()[0][0]
ratio = round(patient_bmi_count/patient_all_count, 3) * 100

logger.log(f"Total num of patients {patient_all_count}.")
logger.log(f"Found {patient_bmi_count} patients ({ratio}%) with {bmi_count} BMI-values.")

{"type": "info", "time": 1657173626148, "message": "Total num of patients 1326.", "params": null}
{"type": "info", "time": 1657173626149, "message": "Found 450 patients (33.900000000000006%) with 3539 BMI-values.", "params": null}


In [10]:
cursor = connection.cursor()

In [11]:
# print tables of the database
cursor.execute('SELECT name FROM sqlite_master where type="table"') 
print(cursor.fetchall())

[('patients',), ('careplans',), ('conditions',), ('observations',)]


# Star Schema

#### Erstellen Dimensionstabellen

Tabelle patients_

In [12]:
# table patients_
cursor.execute('''DROP TABLE IF EXISTS patients_;''')
cursor.execute('''
        CREATE TABLE patients_ ( 
        ID STRING PRIMARY KEY UNIQUE,
        RACE STRING, 
        ETHNICITY STRING,
        GENDER STRING,
        AGE INT64
        );''')

In [13]:
# create df patients
cursor.execute('''SELECT ID, BIRTHDATE, DEATHDATE, RACE, ETHNICITY, GENDER FROM PATIENTS;''')
df_patients = pd.DataFrame(cursor.fetchall(), columns=['ID', 'BIRTHDATE', 'DEATHDATE', 'RACE', 'ETHNICITY', 'GENDER'])

In [14]:
# convert to date
df_patients["DEATHDATE"] = pd.to_datetime(df_patients["DEATHDATE"])
df_patients["BIRTHDATE"] = pd.to_datetime(df_patients["BIRTHDATE"])
# fill null values withh todays date
df_patients['DEATHDATE'] = df_patients.DEATHDATE.fillna(pd.to_datetime("today"))
# calculate age
df_patients["AGE"] = df_patients.DEATHDATE.dt.year - df_patients.BIRTHDATE.dt.year
# drop unnecessary variables
df_patients = df_patients.drop(['BIRTHDATE', 'DEATHDATE'], axis=1)

In [15]:
df_patients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1326 entries, 0 to 1325
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   ID         1326 non-null   object
 1   RACE       1326 non-null   object
 2   ETHNICITY  1326 non-null   object
 3   GENDER     1326 non-null   object
 4   AGE        1326 non-null   int64 
dtypes: int64(1), object(4)
memory usage: 51.9+ KB


In [16]:
print("Number of Duplicated Rows", df_patients.duplicated(df_patients.columns).sum())

Number of Duplicated Rows 0


In [17]:
df_patients.to_sql('df_patients', connection, if_exists='replace', index=False)
cursor.execute('INSERT INTO patients_ (ID, RACE, ETHNICITY, GENDER, AGE) SELECT ID, RACE, ETHNICITY, GENDER, RACE FROM df_patients;')
cursor.execute('''DROP TABLE IF EXISTS df_patients;''')

print(pd.read_sql_query("PRAGMA table_info(patients_)", connection))

   cid       name    type  notnull dflt_value  pk
0    0         ID  STRING        0       None   1
1    1       RACE  STRING        0       None   0
2    2  ETHNICITY  STRING        0       None   0
3    3     GENDER  STRING        0       None   0
4    4        AGE   INT64        0       None   0


Tabelle observations_

In [18]:
# table observations_
cursor.execute('''DROP TABLE IF EXISTS observations_;''')
cursor.execute('''
        CREATE TABLE observations_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING,
        UNITS STRING, 
        TYPE STRING
        );''')

# create df observation
cursor.execute('''SELECT CODE, DESCRIPTION, UNITS, TYPE FROM OBSERVATIONS;''')
df_observations = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION', 'UNITS', 'TYPE'])

df_observations = df_observations.drop_duplicates(subset='CODE')

# transform dt in table
df_observations.to_sql('df_observations', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO observations_ (CODE, DESCRIPTION, UNITS, TYPE) SELECT CODE, DESCRIPTION, UNITS, TYPE FROM df_observations;''')

cursor.execute('''DROP TABLE IF EXISTS df_observations;''')
print(pd.read_sql_query("PRAGMA table_info(observations_)", connection))
#print(pd.read_sql_query("SELECT * FROM observations_", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0
2    2        UNITS  STRING        0       None   0
3    3         TYPE  STRING        0       None   0


Tabelle careplans_code

In [19]:
# table careplans_code
cursor.execute('''DROP TABLE IF EXISTS careplans_code;''')
cursor.execute('''
        CREATE TABLE careplans_code ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT CODE, DESCRIPTION FROM CAREPLANS;''')
df_careplans_code = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_careplans_code = df_careplans_code.drop_duplicates(subset='CODE')

# transform dt in table
df_careplans_code.to_sql('df_careplans_code', connection, if_exists='replace', index=False)

cursor.execute('''INSERT INTO careplans_code (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_careplans_code;''')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_code;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_code)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_code", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0


Tabelle careplans_reasoncode

In [20]:
# table careplans_reasoncode
cursor.execute('''DROP TABLE IF EXISTS careplans_reasoncode;''')
cursor.execute('''
        CREATE TABLE careplans_reasoncode ( 
        REASONCODE STRING PRIMARY KEY UNIQUE,
        REASONDESCRIPTION STRING
        );''')

# create df careplans_code
cursor.execute('''SELECT REASONCODE, REASONDESCRIPTION FROM CAREPLANS;''')
df_careplans_reasoncode = pd.DataFrame(cursor.fetchall(), columns=['REASONCODE','REASONDESCRIPTION'])

df_careplans_reasoncode = df_careplans_reasoncode.drop_duplicates(subset='REASONCODE')

# transform dt in table
df_careplans_reasoncode.to_sql('df_careplans_reasoncode', connection, if_exists='replace', index=False)
   
cursor.execute('INSERT INTO careplans_reasoncode (REASONCODE, REASONDESCRIPTION) SELECT REASONCODE, REASONDESCRIPTION FROM df_careplans_reasoncode;')

cursor.execute('''DROP TABLE IF EXISTS df_careplans_reasoncode;''')
print(pd.read_sql_query("PRAGMA table_info(careplans_reasoncode)", connection))
#print(pd.read_sql_query("SELECT * FROM careplans_reasoncode", connection))

   cid               name    type  notnull dflt_value  pk
0    0         REASONCODE  STRING        0       None   1
1    1  REASONDESCRIPTION  STRING        0       None   0


Tabelle conditions_

In [21]:
# table conditions
cursor.execute('''DROP TABLE IF EXISTS conditions_;''')
cursor.execute('''
        CREATE TABLE conditions_ ( 
        CODE STRING PRIMARY KEY UNIQUE,
        DESCRIPTION STRING
        );''')

# create df conditions
cursor.execute('''SELECT CODE, DESCRIPTION FROM CONDITIONS;''')
df_conditions = pd.DataFrame(cursor.fetchall(), columns=['CODE','DESCRIPTION'])

df_conditions = df_conditions.drop_duplicates(subset='CODE')

# transform dt in table
df_conditions.to_sql('df_conditions', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO conditions_ (CODE, DESCRIPTION) SELECT CODE, DESCRIPTION FROM df_conditions;')

cursor.execute('''DROP TABLE IF EXISTS df_conditions;''')
print(pd.read_sql_query("PRAGMA table_info(conditions_)", connection))
#print(pd.read_sql_query("SELECT * FROM conditions_", connection))

   cid         name    type  notnull dflt_value  pk
0    0         CODE  STRING        0       None   1
1    1  DESCRIPTION  STRING        0       None   0


Datumstabelle

In [22]:
from datetime import datetime
def create_date_table(start='1900-01-01', end=datetime.today().strftime('%Y-%m-%d')):
    
    df_date = pd.DataFrame({"Date": pd.date_range(start, end)})

    days_names = {
        i: name
        for i, name
        in enumerate(['Monday', 'Tuesday', 'Wednesday',
                      'Thursday', 'Friday', 'Saturday', 
                      'Sunday'])
    }
   
    df_date["Day"] = df_date.Date.dt.dayofweek.map(days_names.get)
    df_date["Week"] = df_date.Date.dt.weekofyear
    df_date["Quarter"] = df_date.Date.dt.quarter
    df_date["Year"] = df_date.Date.dt.year
    df_date["Year_half"] = (df_date.Quarter + 1) // 2
    
    return df_date

In [23]:
# table date_table
cursor.execute('''DROP TABLE IF EXISTS date_table;''')
cursor.execute('''
        CREATE TABLE date_table ( 
        DATE DATE PRIMARY KEY UNIQUE,
        DAY STRING,
        WEEK INT16,
        QUARTER INT16,
        YEAR INT16,
        YEAR_HALF INT16
        );''')

In [24]:
df_date = create_date_table()

# transform dt in table
df_date.to_sql('df_date', connection, if_exists='replace', index=False)

cursor.execute('INSERT INTO date_table (DATE, DAY, WEEK, QUARTER, YEAR, YEAR_HALF) SELECT DATE, DAY, WEEK, QUARTER, YEAR, YEAR_HALF FROM df_date;')

cursor.execute('''DROP TABLE IF EXISTS df_date;''')
print(pd.read_sql_query("PRAGMA table_info(date_table)", connection))

C:\Users\Sofia Henninger\AppData\Local\Temp\ipykernel_4460\3083742411.py:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  df_date["Week"] = df_date.Date.dt.weekofyear


   cid       name    type  notnull dflt_value  pk
0    0       DATE    DATE        0       None   1
1    1        DAY  STRING        0       None   0
2    2       WEEK   INT16        0       None   0
3    3    QUARTER   INT16        0       None   0
4    4       YEAR   INT16        0       None   0
5    5  YEAR_HALF   INT16        0       None   0


#### Erstelle Faktentabelle

In [25]:
cursor.execute('''DROP TABLE IF EXISTS fact_table;''')
cursor.execute('''
        CREATE TABLE fact_table ( 
        PATIENT_ID STRING,
        OBSERVATION_CODE STRING,
        VALUE STRING,
        DATE DATE,
        CONDITIONS_CODE STRING,
        ENDDATE DATE,
        CAREPLANS_CODE STRING,
        CAREPLANS_REASONCODE STRING,
        FOREIGN KEY (PATIENT_ID)
            REFERENCES patients_(ID),
        FOREIGN KEY (OBSERVATION_CODE)
            REFERENCES observations_(CODE),
        FOREIGN KEY (CONDITIONS_CODE)
            REFERENCES conditions_(CODE),
        FOREIGN KEY (CAREPLANS_CODE)
            REFERENCES careplans_code(CODE),
        FOREIGN KEY (CAREPLANS_REASONCODE)
            REFERENCES careplans_reasoncode(REASONCODE),
        FOREIGN KEY (DATE)
            REFERENCES date_table(DATE),
        FOREIGN KEY (ENDDATE)
            REFERENCES date_table(DATE)
        );''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, OBSERVATION_CODE, VALUE, DATE) 
                    SELECT PATIENT, CODE, VALUE, DATE 
                    FROM OBSERVATIONS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CAREPLANS_REASONCODE, DATE, ENDDATE) 
                    SELECT PATIENT, REASONCODE, START, STOP 
                    FROM CAREPLANS
                    ;''')

cursor.execute('''INSERT INTO fact_table        
                    (PATIENT_ID, CONDITIONS_CODE, DATE, ENDDATE) 
                    SELECT PATIENT, CODE, START, STOP 
                    FROM CONDITIONS
                    ;''')

In [26]:
print(pd.read_sql_query("PRAGMA foreign_key_list(fact_table)", connection))

   id  seq                 table                  from          to  on_update  \
0   0    0            date_table               ENDDATE        DATE  NO ACTION   
1   1    0            date_table                  DATE        DATE  NO ACTION   
2   2    0  careplans_reasoncode  CAREPLANS_REASONCODE  REASONCODE  NO ACTION   
3   3    0        careplans_code        CAREPLANS_CODE        CODE  NO ACTION   
4   4    0           conditions_       CONDITIONS_CODE        CODE  NO ACTION   
5   5    0         observations_      OBSERVATION_CODE        CODE  NO ACTION   
6   6    0             patients_            PATIENT_ID          ID  NO ACTION   

   on_delete match  
0  NO ACTION  NONE  
1  NO ACTION  NONE  
2  NO ACTION  NONE  
3  NO ACTION  NONE  
4  NO ACTION  NONE  
5  NO ACTION  NONE  
6  NO ACTION  NONE  


In [27]:
cursor.execute('SELECT name FROM sqlite_master where type="table"')
print(cursor.fetchall())

[('patients',), ('careplans',), ('conditions',), ('observations',), ('patients_',), ('observations_',), ('careplans_code',), ('careplans_reasoncode',), ('conditions_',), ('date_table',), ('fact_table',)]


## Analysis

## Aufräumen & Logs speichern

In [38]:
connection.close()
#logger.logTimings()
#logger.writeToFile("../artefacts-for-release/analysis-log.json")